In [42]:
import numpy as np
import pandas as pd
from scipy import misc

import os
import shutil
from tqdm import tqdm 

from PIL import Image

In [43]:
# the folder for images
dir_train = 'E:/Jupyter NB/10 class Classification/Images'

# the directory of separeated images
image_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test'

# This cell is for testing

In [44]:
'''
for root, dirs, files in os.walk(dir_train):
    print('---root---')
    print(root)
    print('---dirs---')
    print(dirs)
    print('---files---')
    print(files)
    print('---------------------')



subdirs = list(os.walk(dir_train))[1:]

for dir_path, _, files in tqdm(subdirs):
    
    dir_name = dir_path.split('\\')[-1]
    print(dir_name)
'''

"\nfor root, dirs, files in os.walk(dir_train):\n    print('---root---')\n    print(root)\n    print('---dirs---')\n    print(dirs)\n    print('---files---')\n    print(files)\n    print('---------------------')\n\n\n\nsubdirs = list(os.walk(dir_train))[1:]\n\nfor dir_path, _, files in tqdm(subdirs):\n    \n    dir_name = dir_path.split('\\')[-1]\n    print(dir_name)\n"

In [45]:
subdirs = list(os.walk(dir_train))[1:]

# collect train metadata
train_metadata = []

for dir_path, _, files in tqdm(subdirs):
    
    dir_name = dir_path.split('\\')[-1]
    
    for file_name in files:
        if not file_name.startswith('.'):
            # read image
            temp = misc.imread(os.path.join(dir_path, file_name)) 
            # collect image metadata
            image_metadata = []
            image_metadata.extend([dir_name, file_name])
            image_metadata.extend( 
                list(temp.shape) if len(temp.shape) == 3 
                else [temp.shape[0], temp.shape[1], 1]
            )
            image_metadata.extend([temp.nbytes, temp.dtype])
            # append image metadata to list
            train_metadata.append(image_metadata)

#print(train_metadata)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]C:\Users\aRnob\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:07<00:00,  2.67it/s]


In [46]:
M = pd.DataFrame(train_metadata)
M.columns = ['directory', 'img_name', 'height', 'width', 'channels', 'byte_size', 'bit_depth']

M['category_name'] = M.directory.apply(lambda x: x.split('.')[-1].lower())
M['img_extension'] = M.img_name.apply(lambda x: x.split('.')[-1])
M['category_number'] = M.directory.apply(lambda x: int(x.split('.')[0]))

In [47]:
# number of grayscale images
(M.channels != 3).sum()

12

In [48]:
# number of categories
M.category_name.nunique()

20

In [49]:
# class number -> class name
decode = {n: i for i, n in M.groupby('category_name').category_number.first().iteritems()}
decode

np.save('decode.npy', decode)

In [50]:
# 20 images per class
V = M.groupby('category_name', group_keys=False).apply(lambda x: x.sample(n=20, replace=False))
V.sort_index(inplace=True)
M.drop(V.index, axis=0, inplace=True)

'''
# 20 images per class for test
Test = M.groupby('category_name', group_keys=False).apply(lambda x: x.sample(n=20, replace=False))
Test.sort_index(inplace=True)
M.drop(Test.index, axis=0, inplace=True)
'''

"\n# 20 images per class for test\nTest = M.groupby('category_name', group_keys=False).apply(lambda x: x.sample(n=20, replace=False))\nTest.sort_index(inplace=True)\nM.drop(Test.index, axis=0, inplace=True)\n"

In [51]:
# train data
len(M)

2357

In [52]:
# validation data
len(V)

400

In [53]:
#Test data
#len(Test)

In [54]:
M.to_csv('train_metadata.csv', index=False)
V.to_csv('val_metadata.csv', index=False)
#Test.to_csv('test_data.csv', index=False)

In [55]:
t = pd.read_csv('E:/Jupyter NB/10 class Classification/train_metadata.csv')
v = pd.read_csv('E:/Jupyter NB/10 class Classification/val_metadata.csv')
#test = pd.read_csv('E:/Jupyter NB/10 class Classification/test_data.csv')

In [56]:
os.mkdir(image_dir + '/train_no_resizing')
for i in range(1, 20 + 1):
    os.mkdir(image_dir + '/train_no_resizing/' + str(i))

In [57]:
os.mkdir(image_dir + '/val_no_resizing')
for i in range(1, 20 + 1):
    os.mkdir(image_dir + '/val_no_resizing/' + str(i))

In [58]:
'''
os.mkdir(image_dir + '/test_no_resizing')
for i in range(1, 20 + 1):
    os.mkdir(image_dir + '/test_no_resizing/' + str(i))
'''

"\nos.mkdir(image_dir + '/test_no_resizing')\nfor i in range(1, 20 + 1):\n    os.mkdir(image_dir + '/test_no_resizing/' + str(i))\n"

# Train images

In [59]:
train_size = len(t)
train_size

2357

In [60]:
# RGB images
for i, row in tqdm(t.loc[t.channels == 3].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # save
    save_path = os.path.join(image_dir, 'train_no_resizing', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

2347it [00:14, 158.40it/s]


In [61]:
# grayscale images
for i, row in tqdm(t.loc[t.channels == 1].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # convert to RGB
    array = np.asarray(image, dtype='uint8')
    array = np.stack([array, array, array], axis=2)
    image = Image.fromarray(array)
    
    # save
    save_path = os.path.join(image_dir, 'train_no_resizing', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

10it [00:01,  5.23it/s]


# Validation images

In [62]:
val_size = len(v)
val_size

400

In [63]:
# RGB images
for i, row in tqdm(v.loc[v.channels == 3].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # save
    save_path = os.path.join(image_dir, 'val_no_resizing', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

398it [00:02, 145.07it/s]


In [64]:
# grayscale images
for i, row in tqdm(v.loc[v.channels == 1].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # convert to RGB
    array = np.asarray(image, dtype='uint8')
    array = np.stack([array, array, array], axis=2)
    image = Image.fromarray(array)
    
    # save
    save_path = os.path.join(image_dir, 'val_no_resizing', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

2it [00:00, 101.08it/s]


# Test images

In [65]:
'''
test_size = len(test)
test_size
'''

'\ntest_size = len(test)\ntest_size\n'

In [66]:
'''
# RGB images
for i, row in tqdm(test.loc[test.channels == 3].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # save
    save_path = os.path.join(image_dir, 'test_no_resizing', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')
'''

"\n# RGB images\nfor i, row in tqdm(test.loc[test.channels == 3].iterrows()):\n    # get image\n    file_path = os.path.join(dir_train, row.directory, row.img_name)\n    image = Image.open(file_path)\n    \n    # save\n    save_path = os.path.join(image_dir, 'test_no_resizing', str(row.category_number), row.img_name)\n    image.save(save_path, 'jpeg')\n"

In [67]:
'''
# grayscale images
for i, row in tqdm(test.loc[test.channels == 1].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # convert to RGB
    array = np.asarray(image, dtype='uint8')
    array = np.stack([array, array, array], axis=2)
    image = Image.fromarray(array)
    
    # save
    save_path = os.path.join(image_dir, 'test_no_resizing', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')
'''

"\n# grayscale images\nfor i, row in tqdm(test.loc[test.channels == 1].iterrows()):\n    # get image\n    file_path = os.path.join(dir_train, row.directory, row.img_name)\n    image = Image.open(file_path)\n    \n    # convert to RGB\n    array = np.asarray(image, dtype='uint8')\n    array = np.stack([array, array, array], axis=2)\n    image = Image.fromarray(array)\n    \n    # save\n    save_path = os.path.join(image_dir, 'test_no_resizing', str(row.category_number), row.img_name)\n    image.save(save_path, 'jpeg')\n"